In [ ]:
import pandas as pd
import os
import gzip
import pickle
import openai
import torch.nn.functional as F
import torch
import re
import copy
from tqdm import tqdm


In [ ]:
from hardware_mapping import map_hardware_to_table
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

In [ ]:
from hardware_mapping2 import map_hardware_to_table2
# apply mapping2 where hardware_mapped is "not found"
for index, row in hardware_instances.iterrows():
    if row["hardware_mapped"] == "not found":
        hardware_instances.at[index, "hardware_mapped"] = map_hardware_to_table2(row["hardware"])

In [ ]:
for index, row in hardware_instances.iterrows():
    if row["hardware_mapped"] == "bfl single 'sc'" and row["date"] < "2013-04-01":
        hardware_instances.at[index, "hardware_mapped"] = "bitforce sha256 single"

In [ ]:
hardware_instances.head(10)

In [ ]:
len(hardware_instances)

In [ ]:
efficiency = pd.read_csv('../../hardwarelist/hardware_merged.csv') # columns: hardware_name,Mhash/J
efficiency = efficiency.rename(columns={"hardware_name":"hardware_mapped"})

efficiency.head(2)

In [ ]:
joined = hardware_instances.merge(efficiency, on="hardware_mapped", how="left")
joined = joined.dropna(subset=["Mhash/J"])
joined.head(4)

In [ ]:
print(len(joined))

In [ ]:
table = joined[["date","hardware_mapped","Mhash/J"]]
table = table.rename(columns={"hardware_mapped":"hardware_name"})
table["Mhash/J"] = table["Mhash/J"].astype(float).map(lambda x: x/1000000).map(lambda x: f"{x:.10f}")
table = table.rename(columns={"Mhash/J":"TH/J"})
table = table.sort_values(["date","hardware_name"])
table = table.reset_index(drop=True)
table.head(8)

In [ ]:
import numpy as np
# all of these are manually verified to be random noobs trying to use a gpu during the asic era
for index, row in table.iterrows():
    eff = float(row["TH/J"])
    if row["date"] > "2015-07-01" and np.log10(eff*1000000) < 1.5:
        print("a")
        # delete the row
        table = table.drop(index)

In [ ]:
table.to_csv("hardware_instances_with_efficiency.csv", index=False)

In [ ]:
df = pd.read_csv('hardware_instances_with_efficiency.csv') # date,hardware_name,TH/J

max_efficiency = pd.read_csv('../../hardwarelist/Bitcoin max updated2.csv') # rows are date,max (TH/J),archaicity (TH/J)

# max_efficiency = pd.read_csv('../../hardwarelist/pmaxv2.csv') #  date	max_efficiency
# max_efficiency = max_efficiency.rename(columns={"max_efficiency":"max (TH/J)"})


def get_max_efficiency(date):
    date = str(date)[:10]
    try:
        return max_efficiency[max_efficiency['date'] == date]['max (TH/J)'].values[0]
    except:
        return -1

# # for each instance, delete it if it is more efficient than the max efficiency at that time
# for index, row in df.iterrows():
#     eff = float(row["TH/J"])
#     max_eff = get_max_efficiency(row["date"])
#     if eff > max_eff:
#         df = df.drop(index)

df['max_efficiency'] = df['date'].apply(lambda x: get_max_efficiency(x))

In [ ]:
# Convert the 'date' column to a datetime format
df['date'] = pd.to_datetime(df['date'])

# Extract year from the 'date' column and create a new column 'year'
df['year'] = df['date'].dt.year
df

In [ ]:
max_efficiency['date'] = pd.to_datetime(max_efficiency['date'])
max_efficiency = max_efficiency.set_index('date')
max_efficiency

In [ ]:
df.to_csv('plotdata.csv', index=False)